In [1]:
import math

In [3]:
n_A = 4
n_B = 6
total = n_A + n_B

In [4]:
# calculate the proportions
p_A = n_A / total
p_B = n_B / total

In [5]:
# calculation entropy
entropy = -p_A * math.log2(p_A) - p_B * math.log2(p_B)
entropy

0.9709505944546686

In [6]:
# calculate the gini impurity
gini = 1 - p_A**2 - p_B**2
gini

0.48

In [7]:
# calculate the information gain
info_gain = 1 - gini
info_gain

0.52

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [12]:
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [13]:
df.drop("deck", axis=1, inplace=True)

In [14]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True


In [17]:
# impute age with median
imputer = SimpleImputer(strategy='median')
df['age'] = imputer.fit_transform(df[['age']])

In [26]:
round(df.isnull().sum()/len(df)*100, 2)

survived       0.00
pclass         0.00
sex            0.00
age            0.00
sibsp          0.00
parch          0.00
fare           0.00
embarked       0.22
class          0.00
who            0.00
adult_male     0.00
embark_town    0.22
alive          0.00
alone          0.00
dtype: float64

In [35]:
# impute the missing values in embarked and embark town using SimpleImputer

imputer2 = SimpleImputer(strategy='most_frequent')
df[['embarked', 'embark_town']] = imputer2.fit_transform(df[['embarked', 'embark_town']])


In [36]:
df.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     891 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  891 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 79.4+ KB


In [40]:
le = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object' or df[col].dtype == 'category':
        df[col] = le.fit_transform(df[col])


In [41]:
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,1,22.0,1,0,7.2500,2,2,1,True,2,0,False
1,1,1,0,38.0,1,0,71.2833,0,0,2,False,0,1,False
2,1,3,0,26.0,0,0,7.9250,2,2,2,False,2,1,True
3,1,1,0,35.0,1,0,53.1000,2,0,2,False,2,1,False
4,0,3,1,35.0,0,0,8.0500,2,2,1,True,2,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,2,1,1,True,2,0,True
887,1,1,0,19.0,0,0,30.0000,2,0,2,False,2,1,True
888,0,3,0,28.0,1,2,23.4500,2,2,2,False,2,0,False
889,1,1,1,26.0,0,0,30.0000,0,0,1,True,0,1,True


In [42]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')

In [72]:
X = df.drop(["survived", "alive"], axis=1)
y = df['survived']

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [74]:
X_train.shape[0], X_test.shape[0]

(712, 179)

In [75]:
round(X_train.shape[0] / (X_train.shape[0]+X_test.shape[0]), 2)*100

80.0

In [76]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [77]:
y_pred = model.predict(X_test)

In [78]:
confusion_matrix(y_test, y_pred)

array([[80, 25],
       [18, 56]], dtype=int64)

In [79]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.82      0.76      0.79       105
           1       0.69      0.76      0.72        74

    accuracy                           0.76       179
   macro avg       0.75      0.76      0.76       179
weighted avg       0.76      0.76      0.76       179



In [80]:
# save the decision tree classifier using export_graphviz
from sklearn.tree import export_graphviz

export_graphviz(model, out_file='decision_tree.dot', feature_names=X.columns, class_names=['died', 'survived'], filled=True)
